# Preprocessing part


In [22]:
!pip install adversarial-robustness-toolbox

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
import time
import numpy as np 
import torch.nn as nn 
from datetime import datetime

from PIL import Image
from torchvision import transforms
import torchvision
import torch
from torchvision import datasets, transforms
import torch.optim as optim
import torchvision.models as models
import matplotlib.pyplot as plt 

from art.estimators.classification import PyTorchClassifier
from skimage.metrics import structural_similarity as ssim

import warnings
warnings.filterwarnings('ignore') 
import torch

from sklearn.datasets import fetch_lfw_people
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import fetch_lfw_people
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.utils.fixes import loguniform

In [24]:
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)

In [25]:
# introspect the images arrays to find the shapes (for plotting)
n_samples, h, w = lfw_people.images.shape

# for machine learning we use the 2 data directly (as relative pixel
# positions info is ignored by this model)
X = lfw_people.data
n_features = X.shape[1]

# the label to predict is the id of the person
y = lfw_people.target
target_names = lfw_people.target_names
n_classes = target_names.shape[0]

print("Total dataset size:")
print("n_samples: %d" % n_samples)
print("n_features: %d" % n_features)
print("n_classes: %d" % n_classes)

Total dataset size:
n_samples: 1288
n_features: 1850
n_classes: 7


In [26]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [67]:
n_components = 150

print(
    "Extracting the top %d eigenfaces from %d faces" % (n_components, X_train.shape[0])
)
t0 = time.time()
pca = PCA(n_components=n_components, svd_solver="randomized", whiten=True).fit(X_train)
print("done in %0.3fs" % (time.time() - t0))

eigenfaces = pca.components_.reshape((n_components, h, w))

print("Projecting the input data on the eigenfaces orthonormal basis")
t0 = time.time()

X_train_pca = torch.Tensor(pca.transform(X_train))
X_test_pca = torch.Tensor(pca.transform(X_test))


# scaler = StandardScaler()
# X_train_pca = scaler.fit_transform(X_train_pca)
# X_test_pca = scaler.transform(X_test_pca)


# X_train_pca = torch.Tensor(X_train_pca)
# X_test_pca = torch.Tensor(X_test_pca)

print("done in %0.3fs" % (time.time() - t0))

Extracting the top 150 eigenfaces from 966 faces
done in 0.915s
Projecting the input data on the eigenfaces orthonormal basis
done in 0.086s


In [68]:
(n_components, h, w)

(150, 50, 37)

In [69]:
X_train_pca.shape

torch.Size([966, 150])

In [70]:
# create dataset and dataloader
batch_size = 64

train_loader = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(
        torch.from_dlpack(X_train_pca).float(),
        torch.from_dlpack(y_train).long()
    ),
    batch_size=batch_size,
    shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(
        torch.from_dlpack(X_test_pca).float(),
        torch.from_dlpack(y_test).long()
    ),
    batch_size=batch_size,
    shuffle=False
)

In [31]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = nn.Sequential(
    nn.Linear(n_components, 2048),
    nn.ReLU(),
    nn.Linear(2048, 2048),
    nn.ReLU(),
    nn.Linear(2048, n_classes),
    nn.LogSoftmax(dim=1)
)
model = model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = nn.NLLLoss()

In [32]:
# model.load_state_dict(torch.load('custop_model_85.ckpt'))

In [ ]:
epoch=1
model.train()
for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)
    print(batch_idx, (data.shape, target.shape))
    optimizer.zero_grad()
    output = model(data)
    print(output.shape, target.shape)
    print("\t", output[0])
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()
    # if batch_idx % 10 == 0:
    #     print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
    #         epoch, batch_idx * len(data), len(train_loader.dataset),
    #         100. * batch_idx / len(train_loader), loss.item()))

In [34]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        print(batch_idx, (data.shape, target.shape))
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [35]:
epochs = 10
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

0 (torch.Size([64, 150]), torch.Size([64]))
Train Epoch: 1 [0/966 (0%)]	Loss: 1.754302
1 (torch.Size([64, 150]), torch.Size([64]))
2 (torch.Size([64, 150]), torch.Size([64]))
3 (torch.Size([64, 150]), torch.Size([64]))
4 (torch.Size([64, 150]), torch.Size([64]))
5 (torch.Size([64, 150]), torch.Size([64]))
6 (torch.Size([64, 150]), torch.Size([64]))
7 (torch.Size([64, 150]), torch.Size([64]))
8 (torch.Size([64, 150]), torch.Size([64]))
9 (torch.Size([64, 150]), torch.Size([64]))
10 (torch.Size([64, 150]), torch.Size([64]))
Train Epoch: 1 [640/966 (62%)]	Loss: 1.701007
11 (torch.Size([64, 150]), torch.Size([64]))
12 (torch.Size([64, 150]), torch.Size([64]))
13 (torch.Size([64, 150]), torch.Size([64]))
14 (torch.Size([64, 150]), torch.Size([64]))
15 (torch.Size([6, 150]), torch.Size([6]))

Test set: Average loss: 0.0322, Accuracy: 146/322 (45%)

0 (torch.Size([64, 150]), torch.Size([64]))
Train Epoch: 2 [0/966 (0%)]	Loss: 1.883963
1 (torch.Size([64, 150]), torch.Size([64]))
2 (torch.Size(

# Adversarial part

In [36]:
!pip install torchattacks==3.3.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [37]:
import torchattacks

In [38]:
class Attack:
    def __init__(self, model, device, attack_type):
        self.model = model
        self.device = device
        self.attack_type = attack_type
        self.attack = None

    def __call__(self, x, y):
        if self.attack_type == 'FGSM':
            self.attack = torchattacks.FGSM(self.model, eps=8/255)
        elif self.attack_type == 'PGD':
            self.attack = torchattacks.PGD(self.model, eps=8/255, alpha=2/255, steps=25)
        elif self.attack_type == 'CW':
            self.attack = torchattacks.CW(self.model, c=1, kappa=0, steps=25, lr=0.01)
        elif self.attack_type == 'DeepFool':
            self.attack = torchattacks.DeepFool(self.model, steps=25, overshoot=0.02)
        elif self.attack_type == 'BIM':
            self.attack = torchattacks.BIM(self.model, eps=8/255, alpha=2/255, steps=25)
        elif self.attack_type == 'RFGSM':
            self.attack = torchattacks.RFGSM(self.model, eps=8/255, alpha=2/255, steps=25)

        x_adv = self.attack(x, y)
        return x_adv

In [39]:
test_loader_adv = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(
        torch.from_dlpack(x_adv).float(),
        torch.from_dlpack(y_train).long()
    ),
    batch_size=batch_size,
    shuffle=False
)

NameError: ignored

In [65]:
# attack
def transform_data(X):
    X_rescaled = (X - X.min()) / (X.max() - X.min())
    X_rescaled = torch.clamp(X_rescaled, min=0, max=1)
    # print(X_rescaled.numpy())
    # .clone().detach().long()
    return X_rescaled

In [74]:
def attack(model, device, method, loader) -> torch.Tensor:
  attack = Attack(model, device, method)
  x_adv = []
  for batch_idx, (data, target) in enumerate(loader):
      data, target = data.to(device), target.to(device)
      x_adv.append(attack(data, target))
  x_adv = torch.cat(x_adv, dim=0)
  test_loader_adv = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(
        torch.from_dlpack(x_adv).float(),
        torch.from_dlpack(y_test).long()
    ),
    batch_size=batch_size,
    shuffle=False
  )
  test(model, device, test_loader_adv)
  return x_adv
attack(model, device, "FGSM", test_loader)
# attack(model, device, "PGD", test_loader)
# attack(model, device, "CW", test_loader)
# attack(model, device, "DeepFool", test_loader)
# attack(model, device, "BIM", test_loader)
# attack(model, device, "RFGSM", test_loader)

ValueError: ignored

In [ ]:
x_adv.shape

In [ ]:
x_adv = []
for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)
    x_adv.append(attack(data, target))
x_adv = torch.cat(x_adv, dim=0)
test(model, device, test_loader_adv)

In [ ]:
# plot adversarial examples
import matplotlib.pyplot as plt
import numpy as np


def show_images_adversarial(images, cls_true, cls_pred=None):
    assert len(images) == len(cls_true)
    fig, axes = plt.subplots(3, 3)
    fig.subplots_adjust(hspace=0.3, wspace=0.3)

    for i, ax in enumerate(axes.flat):
        ax.imshow(images[i], cmap='gray')
        if cls_pred is None:
            xlabel = "True: {0}".format(cls_true[i])
        else:
            xlabel = "True: {0}, Pred: {1}".format(cls_true[i], cls_pred[i])

        ax.set_xlabel(xlabel)
        ax.set_xticks([])
        ax.set_yticks([])
    plt.show()